In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter

In [4]:
COMPANY_IDENTIFIER = ['GF']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

## Check download status summaries

In [6]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [7]:
license_numbers

array(['C12-0000191-LIC', 'C11-0000401-LIC', 'C10-0000805-LIC',
       'C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000117-LIC',
       'C10-0000224-LIC'], dtype=object)

In [ ]:
#bad_download_history = client_surveillance_jupyter.check_company_license_download(license_numbers,download_summary_records)

In [ ]:
#bad_download_history

## Choose license numbers

In [8]:
retail_license_numbers = ['C10-0000700-LIC','C10-0000805-LIC','C10-0000824-LIC','C10-0000117-LIC','C10-0000224-LIC']


In [9]:
distro_license_numbers = ['C11-0000401-LIC']

In [10]:
all_license_numbers = ['C12-0000191-LIC', 'C11-0000401-LIC', 'C10-0000805-LIC',
       'C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000117-LIC',
       'C10-0000224-LIC']

## Pull data

## distro inventory

In [13]:
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=distro_license_numbers,
)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
df_inventory = company_inventory_packages_dataframe
df_inventory['license_number'].unique()

array(['C11-0000401-LIC'], dtype=object)

## retail incoming

In [14]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=retail_license_numbers,
)

company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
df_in_retail = company_incoming_transfer_packages_dataframe
df_in_retail['license_number'].unique()

array(['C10-0000824-LIC', 'C10-0000805-LIC', 'C10-0000700-LIC',
       'C10-0000224-LIC'], dtype=object)

In [19]:
df_in_retail_internal = df_in_retail[df_in_retail['shipper_facility_license_number'] == 'C11-0000401-LIC']

## all incoming

In [17]:
all_incoming_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=all_license_numbers,
)

df_in_all = pd.read_sql_query(all_incoming_query, engine)
df_in_all['license_number'].unique()

array(['C11-0000401-LIC', 'C10-0000824-LIC', 'C10-0000805-LIC',
       'C10-0000700-LIC', 'C10-0000224-LIC', 'C12-0000191-LIC'],
      dtype=object)

## distro incoming

In [18]:
distro_incoming_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=distro_license_numbers,
)

df_in_distro = pd.read_sql_query(distro_incoming_query, engine)
df_in_distro['license_number'].unique()

array(['C11-0000401-LIC'], dtype=object)

## retail sales

In [20]:
company_sales_receipts_with_transactions_query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=retail_license_numbers,
)
company_sales_receipts_with_transactions_dataframe = pd.read_sql_query(company_sales_receipts_with_transactions_query, engine)
company_sales_receipts_with_transactions_dataframe['license_number'].unique()



array(['C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000805-LIC',
       'C10-0000117-LIC', 'C10-0000224-LIC'], dtype=object)

## dedupe sales transactions

In [21]:
deduped_sales_receipts_with_transactions_dataframe = prepare_data.dedupe_sales_transactions(company_sales_receipts_with_transactions_dataframe)

In [22]:
deduped_sales_receipts_with_transactions_dataframe['sales_month'] = deduped_sales_receipts_with_transactions_dataframe['sales_datetime'].dt.strftime('%Y-%m')

In [23]:
#check we have all locations
deduped_sales_receipts_with_transactions_dataframe['license_number'].unique()

array(['C10-0000824-LIC', 'C10-0000700-LIC', 'C10-0000805-LIC',
       'C10-0000117-LIC', 'C10-0000224-LIC'], dtype=object)

In [24]:
df_sales_retail = deduped_sales_receipts_with_transactions_dataframe

## Receiver wholesale price coverage 


In [28]:
df_in_retail[df_in_retail['receiver_wholesale_price'].notnull()].shape

(38257, 31)

In [29]:
rwp_coverage = client_surveillance_jupyter.check_receiver_wholesale_price_coverage(df_in_retail)


98.0% of incoming transfer packages have receiver wholesale price


# Inventory

In [30]:
today = date.today()
today

datetime.date(2022, 9, 12)

In [31]:
COMPANY_IDENTIFIER

['GF']

In [32]:
distro_license_numbers

['C11-0000401-LIC']

## 1. distro inventory + sales based (distro to retail)

In [43]:
inv_val_distro_to_retail,inv_all_distro_to_retail = client_surveillance_jupyter.calculate_inventory_valuation(df_in_retail,df_inventory,license_numbers,today)

#make sense for CPG


In [44]:
inv_val_distro_to_retail

,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-09-12,1061293.74,1339883.35,818,1183,0.69,"[C12-0000191-LIC, C11-0000401-LIC, C10-0000805...","FLOR EAST BAY, LLC"


In [35]:
inv_val_distro_to_retail['value'].values / inv_val_distro_to_retail['coverage'].values

array([1538106.8695652175], dtype=object)

In [47]:
missing = inv_all_distro_to_retail[inv_all_distro_to_retail['per_unit_product'].isna()]

In [53]:
missing[['product_name','unit_of_measure','quantity']].groupby(['product_name','unit_of_measure']).sum().sort_values(by = 'quantity')

,,quantity
product_name,unit_of_measure,
LPC x Jealousy 3.5g,Each,1.0
High Supply - Gelato - Indica - Flower 1oz,Each,1.0
High Supply - Ghost Train Haze - Sativa - Flower 1oz,Each,1.0
High Supply - Giesel - Indica - Flower 14g,Each,1.0
CALI HASH | Cherry Pie 1g Raw Hash,Each,1.0
...,...,...
Paletas Strawberry Lemonade Sativa Infused Moonrock Preroll 1g,Each,1027.0
Hum Made Processing - Ice Cream Cake Trimmed Smalls,Grams,2072.0
GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,Each,3048.0


In [ ]:
# search for these products from GF's retail sales
# when they come in? new product?
#retailers should be selling them, do they appear in transactions?


In [51]:
missing

,license_number,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,per_unit_incoming,per_unit_product
0,C11-0000401-LIC,28747266,1A4060300004FB1000123633,active,2022-09-12,2022-09-12 16:22:43+00:00,Product,Almora Farm 1g Jar: Rosin: Wedding Cake [H],Other Concentrate (weight - each),24.0,Each,4346332,Concentrate,,F6000016785PB,"WC-10.02.20-AG0769, WC-10.02.20-AG0769B, WC-10...",False,False,False,None,None,NaN,NaN
1,C11-0000401-LIC,28746687,1A4060300004FB1000121615,active,2022-09-12,2022-09-12 16:14:32+00:00,Product,"Biscotti 3.5 Flower- Sativa- California Limes,...",Flower (packaged eighth - each),32.0,Each,4597580,Buds,,BUDA-30863,"000051 T03 Harvest Bay9 10AUG2021, 000060 T03 ...",False,False,False,None,None,NaN,NaN
5,C11-0000401-LIC,28747305,1A4060300004FB1000121621,active,2022-09-12,2022-09-12 16:23:54+00:00,Product,21176 Coastal Sun Greenhouse Flower 3.5g Jar S...,Flower (packaged eighth - each),64.0,Each,4970716,Buds,,,"22X3,6/29/22",False,False,False,None,None,NaN,NaN
7,C11-0000401-LIC,28747328,1A4060300004FB1000121623,active,2022-09-12,2022-09-12 16:26:19+00:00,Product,20997 Coastal Sun Greenhouse Flower 3.5g Jar S...,Flower (packaged eighth - each),64.0,Each,4945664,Buds,,,2022-07-01 Spy Rock Sour Diesel,False,False,False,None,None,NaN,NaN
13,C11-0000401-LIC,28746797,1A4060300004FB1000121616,active,2022-09-12,2022-09-12 16:18:28+00:00,Product,Biscotti 3.5 Flower- Indica- Cherry Cheesecake...,Flower (packaged eighth - each),64.0,Each,4890479,Buds,,B-1-8-TH-S-D-1,U12 - 012 - 000 - 13822,False,False,False,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,C11-0000401-LIC,21997907,1A4060300004FB1000080772,active,2022-02-04,2022-02-04 18:47:49+00:00,Product,TRADE SAMPLE Pax Tahoe Rose Live Rosin Pod 0.5g,Vape Cartridge (weight - each),50.0,Each,4149422,Concentrate,,210270PTRPGH0283,"2021-05-21-Block 1-H, 2021-05-22-Block 1-H, 20...",False,False,False,None,None,NaN,NaN
1143,C11-0000401-LIC,21819622,1A40603000045EF000023845,active,2022-01-30,2022-08-29 15:35:56+00:00,Product,0.5g Cart - P2 - Sativa - Gelonade,Vape Cartridge (weight - each),10.0,Each,4082925,Concentrate,,011922GLN,"110 - 1594A-09252021, 110-1594A-09252021, 38 -...",False,False,False,None,None,NaN,NaN
1152,C11-0000401-LIC,21623120,1A4060300008D6B000008955,active,2022-01-24,2022-01-28 17:16:48+00:00,Product,CALI HASH | Cherry Pie 1g Raw Hash,Extract (weight - each),1.0,Each,4106761,Concentrate,,CGCP001,PB RM#2 11.8,False,False,False,None,None,NaN,NaN
1179,C11-0000401-LIC,18955488,1A4060300002A31000000953,active,2021-10-21,2021-10-25 18:39:32+00:00,Product,GRIZZLY PEAK - Indica Bone - 0.5g - 7PK,Pre-Roll Infused,3048.0,Each,3573113,Concentrate,IB7PK005,,7/27/21 - 200 Flower 1 - Club Soda - Wet Flowe...,False,False,False,None,None,NaN,NaN


## 2. distro inventory + cost based (cost to distro)

In [36]:
inv_val_distro_cost,inv_all_distro_cost = client_surveillance_jupyter.calculate_inventory_valuation(df_in_distro,df_inventory,license_numbers,today)




In [37]:
inv_val_distro_cost

,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-09-12,436221.72,550729.92,495,1183,0.42,"[C12-0000191-LIC, C11-0000401-LIC, C10-0000805...","FLOR X, INC."


In [38]:
inv_val_distro_cost['value'].values / inv_val_distro_cost['coverage'].values

array([1038623.1428571428], dtype=object)

## 3. blended: distro inventory + all incoming data

In [39]:
inv_val_all,inv_all = client_surveillance_jupyter.calculate_inventory_valuation(df_in_all,df_inventory,license_numbers,today)




In [40]:
inv_val_all

,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-09-12,1081135.49,1364933.55,880,1183,0.74,"[C12-0000191-LIC, C11-0000401-LIC, C10-0000805...","FLOR X, INC."


In [41]:
inv_val_all['value'].values / inv_val_all['coverage'].values

array([1460993.9054054054], dtype=object)